In [1]:
# 載入所需套件
import math 
import tejapi
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from scipy.stats import norm
plt.style.use('bmh')
plt.rcParams['font.sans-serif']=['Microsoft YaHei']

# 登入TEJ API
api_key = 'YOUR_KEY'
tejapi.ApiConfig.api_key = api_key
tejapi.ApiConfig.ignoretz = True

gte, lte = '2021-03-16', '2023-04-10'
# 標的物價格
stocks = tejapi.get('TWN/APRCD',
                   paginate = True,
                   coid = 'Y9999',
                   mdate = {'gte':gte, 'lte':lte},
                   chinese_column_name = True,
                   opts = {
                       'columns':[ 'mdate','close_d']
                   }
                  )
# 選擇權價格
options = tejapi.get(
    'TWN/AOPTION',
    paginate = True,
    coid = 'TXO202304C15500',
    mdate = {'gte':gte, 'lte':lte},
    chinese_column_name = True,
    opts = {
        'columns':['mdate', 'coid','settle', 'kk', 'theoremp', 'acls', 'ex_price', 'td1y', 'avolt', 'rtime']
    }
)
# 重設日期為index
stocks = stocks.set_index('年月日')
options = options.set_index('日期')

stocks['日報酬'] = np.log(stocks['收盤價(元)']) - np.log(stocks['收盤價(元)'].shift(1))
stocks['移動報酬波動度'] = stocks['日報酬'].rolling(252).std()



ModuleNotFoundError: No module named 'tejapi'

In [ ]:
class BS_formula:

    def __init__(self, s0, k, r, sigma, T):     
        self.s0 = s0 # 標的物價格
        self.k = k # 履約價格
        self.r = r # 無風險利率
        self.sigma = sigma # 歷史波動度
        self.T = T # 剩餘到期時間
        self.d1 = (np.log(s0/k)+(r+sigma**2/2)*T) / (sigma * np.sqrt(T))
        self.d2 = ((np.log(s0/k)+(r+sigma**2/2)*T) / (sigma * np.sqrt(T))) - sigma*np.sqrt(T)
        
    def BS_price(self): # 計算理論價格
        c = self.s0*norm.cdf(self.d1) - self.k*np.exp(-self.r*self.T)*norm.cdf(self.d2)
        p = self.k*np.exp(-self.r*self.T)*norm.cdf(-self.d2) - self.s0*norm.cdf(-self.d1)
        return c,p
        
    def BS_delta(self): # 計算 delta
        return norm.cdf(self.d1), norm.cdf(self.d1)-1
    
    def BS_gamma(self): # 計算 gamma
        return norm.pdf(self.d1)/(self.s0*self.sigma*np.sqrt(self.T)), norm.pdf(self.d1)/(self.s0*self.sigma*np.sqrt(self.T))
    
    def BS_vega(self): # 計算 vega
        return self.s0*np.sqrt(self.T)*norm.pdf(self.d1), self.s0*np.sqrt(self.T)*norm.pdf(self.d1)
    
    def BS_theta(self): # 計算 theta 
        c_theta = -self.s0*norm.pdf(self.d1)*self.sigma / (2*np.sqrt(self.T)) - self.r*self.k*np.exp(-self.r*self.T)*norm.cdf(self.d2)
        p_theta = -self.s0*norm.pdf(self.d1)*self.sigma / (2*np.sqrt(self.T)) + self.r*self.k*np.exp(-self.r*self.T)*norm.cdf(-self.d2)
        return c_theta, p_theta
    
    def BS_rho(self): # 計算 rho  
        return self.k*self.T*np.exp(-self.r*self.T)*norm.cdf(self.d2), -self.k*self.T*np.exp(-self.r*self.T)*norm.cdf(-self.d2)

In [ ]:
s0 = np.linspace(200,800)
k = 500
r = 0.00
sigma = 0.2
T = 252/252

mybs = BS_formula(s0, k, r, sigma, T)
c, p = mybs.BS_price()

fig = plt.figure(figsize = (12,8))
plt.plot(s0, c, label = '買權')
plt.plot(s0, p, label = '賣權')
plt.axvline(x = 500, color = 'black', linestyle = '--')
plt.xlabel('標的物價格', fontsize = 15)
plt.ylabel('選擇權價格', fontsize = 15)
plt.title('選擇權價格 VS. 標的物價格', fontsize = 20)
plt.legend(fontsize = 14)
plt.savefig('black scholes put call price.png')
plt.show()

In [ ]:
s0 = np.linspace(200,800)
k = 500
r = 0.00
sigma = 0.2
T = 252/252

mybs = BS_formula(s0, k, r, sigma, T)
c, p = mybs.BS_delta()

fig = plt.figure(figsize = (12,8))
plt.plot(s0, c, label = '買權')
plt.plot(s0, p, label = '賣權')
plt.axvline(x = 500, color = 'black', linestyle = '--')
plt.axhline(y = 0, color = 'black', linestyle = '--')
plt.xlabel('標的物價格', fontsize = 15)
plt.ylabel('Delta值', fontsize = 15)
plt.title('Delta值 VS. 標的物價格', fontsize = 20)
plt.legend(fontsize = 14)
plt.savefig('black scholes put call delta.png')
plt.show()

In [ ]:
s0, s1, s2 = 400, 500, 600 
k = 500
r = 0.00
sigma = 0.2
T = np.linspace(1, 0.01)

mybs0 = BS_formula(s0, k, r, sigma, T)
c0, p0 = mybs0.BS_gamma()

mybs1 = BS_formula(s1, k, r, sigma, T)
c1, p1 = mybs1.BS_gamma()

mybs2 = BS_formula(s2, k, r, sigma, T)
c2, p2 = mybs2.BS_gamma()

fig = plt.figure(figsize = (12,8))
plt.plot(T, c0, label = '買權(價外)')
plt.plot(T, c1, label = '買權(價平)')
plt.plot(T, c2, label = '買權(價內)')
plt.xlabel('剩餘時間', fontsize = 15)
plt.ylabel('Gamma值', fontsize = 15)
plt.title('Gamma值 VS. 剩餘時間', fontsize = 20)
plt.legend(fontsize = 14)
plt.axis([1.005, -0, -0.005, .045])
plt.savefig('black scholes put call gamma2.png')
plt.show()

In [ ]:
r = 0.012
s0 = stocks.loc['2023-04-10']['收盤價(元)']
k = 15500
sigma = stocks.loc['2023-04-10']['移動報酬波動度']*np.sqrt(252)
T = 6/252

mybs = BS_formula(s0, k, r, sigma, T)
c, p = mybs.BS_price()
c_delta, p_delta = mybs.BS_delta()
c_gamma, p_gamma = mybs.BS_gamma()
c_vega, p_vega = mybs.BS_vega()
c_theta, p_theta = mybs.BS_theta()
c_rho, p_rho = mybs.BS_rho()

print('==2023-04-10履約價為525的台積電買權==')
print('當前標的物價格為 %.3f， 年化波動度為 %.3f， 剩餘期間為 %.3f'%(s0, sigma, T*252))
print('買權理論價格: %.4f， 賣權理論價格: %.4f' %(c,p))
print('買權delta: %.4f， 賣權delta: %.4f' %(c_delta,p_delta))
print('買權gamma: %.4f， 賣權gamma: %.4f' %(c_gamma,p_gamma))
print('買權vega: %.4f， 賣權vega: %.4f' %(c_vega,p_vega))
print('買權theta: %.4f， 賣權theta: %.4f' %(c_theta,p_theta))
print('買權rho: %.4f， 賣權rho: %.4f' %(c_rho,p_rho))

In [ ]:
options.loc['2023-04-10'] # 實際買權價格